<a href="https://colab.research.google.com/github/EffiSciencesResearch/ML4G-2.0/blob/master/workshops/agents/agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
# LLM Agents

This notebook is an introduction to the openai & anthropics API and to the design of LLM-agents.

In the first part, your goal will be to make a chatbot that negotiates the price of a specific good with you, then against an other model. This could be part of a persuation benchmark, where we evaluate how well a LLM can drive the price down against an other LLM.


> ## Learning outcomes
> - Knowing how to use LLM API
> - Finding your way in the documentation
> - Understand how to make LLMs take actions
> - Experiment with prompt engineering and control LLM outputs

During the workshop, you will need the documentation 
- For the OpenAI API: https://platform.openai.com/docs/
- For the Anthropics API: https://docs.anthropic.com/claude/docs/

In [ ]:
import os
import json
import openai
import anthropic

openai_key = os.environ.get("OPENAI_API_KEY") or input("OpenAI API Key")
anthropic_key = os.environ.get("ANTHROPIC_API_KEY") or input("Anthropic API Key")

openai_client = openai.Client(api_key=openai_key)
anthropic_client = anthropic.Client(api_key=anthropic_key)

In [ ]:
MODELS = [
    # Small, cheap and fast
    "claude-3-haiku-20240307",
    # Medium
    "gpt-3.5-turbo",
    "claude-3-sonnet-20240229",
    # Big, slow expensive and good
    "gpt-4-turbo-preview",
    "claude-3-opus-20240229",
]

MODEL = MODELS[1]

## Chat human-LLM

We'll start be creating one function to handle all the details of the APIs, so that we can forget about them later and focus more on the logic.

**Important note**: When you develop applications, evaluations or benchmarks with LLMs it is important always test with the smallest model first, as they are much faster and cheaper. This let you do more and faster iterations. However, when you start to tweak prompts, you need to tweak your prompts for one specific LLM, as they all read differently. The best prompt on GPT3 can be quite bad on GPT4 and vice versa.


Start by having the function work for openai's models, test it on the cells bellow, and you can later come back and implement it for anthropic. The anthropic part is especially interesting when we get to make the two of them chat. Who's the most persuasive?


In [ ]:
def generate_answer(system: str, *messages: str, model: str = "gpt-3.5-turbo") -> str:
    """
    Generate the next message from the specified model.

    Args:
        system: the system prompt to use
        messages: the content of all the messages in the conversation, the first
            message is always with the "user" role, then it alternates between
            "assistant" and "user"
        model: the name of the model to use.
    """

    if "gpt" in model:
        # Use openai API
        ...

    elif "claude" in model:
        # Use anthropic API
        # Implement this later, you don't need to APIs at the start.
        ...

    else:
        raise ValueError(f"Unkown model: {model!r}")

Bonus for later: make the API stream the answer, so that you can print it as it is generated. You can either print it directly in the function or transform the function in a generator that yields strings.

In [ ]:
generate_answer(
    "Answer the questions for the user, always in 2 sentences and from the perspective of the french president",
    "What are counterintuitive ways to make the most out of a summer school?",
    model="claude-3-haiku-20240307",
)

Now we need a loop to keep the discussion going and add the new messages to the discussion. 
A few points to have in mind:
- How do you know when to stop the loop? Can it continue forever?
- The messages for the API need to start with a message from the "user". Who is the user here, and how do you generate the first message?
- You may need to add a time.sleep() in the loop to avoid rate limits. Bonus: catch rate limits errors and wait for the exact time.

In [ ]:
VENDOR_PROMPT = r"""
You sell tables. Negotiate for a high price.

... (add more context info about what you want them to do, like ~2 sentences)
... (add instructions for how to make offers and accept them)
"""

BUYER_PROMPT = r"""
You are looking to buy a nice table, for as cheap as possible.

... (add instructions for how to make offers and accept them)
"""

STOP = "Offer accepted!"


def chat_two_llms(
    vendor_system: str,
    buyer_system: str,
    vendor_model: str = MODEL,
    buyer_model: str = MODEL,
    stop: str = None,
):
    """Print a dialoge between the 2 LLMs."""
    ...


chat_two_llms(VENDOR_PROMPT, BUYER_PROMPT, stop=STOP)

At how much was the agreement? Does it change when you change models? Compare with the other people in the room. Are bigger models better at persuation? 

This is the simplest model of chat interaction between two LLMs. In practice, we don't often make them chat to each other, but interesting papers have created [a village of LLMs](https://arxiv.org/abs/2304.03442),
a [virtual game developement company](https://github.com/OpenBMB/ChatDev), or are even using them to [simulate social dynamics](https://arxiv.org/abs/2208.04024) and [model epidemic spread](https://arxiv.org/abs/2307.04986).

Here the LLMs chat directly to each other, but in practice, it is useful to allow them to think before they speak (yes, that's not only true for humans). This means that all of the output of a LLM won't be used in the process, it's only useful to them.
This also means that we need to parse the response of the LLM somehow to find what's addressed to the chat and what's for themselves.

A nice trick is to ask them to output JSON, with keys that you specify, and in the order that you specify. This way you can ensure that the reasoning should come before the message to send for instance, or a reasoning comes before an answer.

In [ ]:
VENDOR_PROMPT = r"""
You sell tables. You inherited all the tables imaginable would like to sell one, but need to sell it for as much as you can.
The person in front of you seems interested in a new table.

Use the following JSON format for your output, without quotes nor comments:
{
    "private reasoning": <str>,
    "message": <str>,
    "offer": <float> or null,
    "offer accepted": true,
}

Important: your goal is to negociate to have the highest final price possible.
"""

BUYER_PROMPT = r"""
You are looking to buy a nice table, for as cheap as possible.

Use the following JSON format for your output, without quotes nor comments:
{
    "private reasoning": <str>,
    "message": <str>,
    "offer": <float> or null,
    "offer accepted": <bool>,
}

Important: your goal is to negociate to pay the lowest final price possible.
"""

STOP = "Offer accepted!"


def chat_two_llms_with_private_reasoning(
    vendor_system: str,
    buyer_system: str,
    vendor_model: str = MODEL,
    buyer_model: str = MODEL,
    stop: str = None,
    max_turns: int = 5,
):
    """Print a dialoge between the 2 LLMs that can think for themselves"""

    ...


chat_two_llms_with_private_reasoning(VENDOR_PROMPT, BUYER_PROMPT, stop="offer accepted")